## LangGraph Open Deep Research - Supervisor-Researcher Architecture

In this notebook, we'll explore the **supervisor-researcher delegation architecture** for conducting deep research with LangGraph.

You can visit this repository to see the original application: [Open Deep Research](https://github.com/langchain-ai/open_deep_research)

Let's jump in!

## What We're Building

This implementation uses a **hierarchical delegation pattern** where:

1. **User Clarification** - Optionally asks clarifying questions to understand the research scope
2. **Research Brief Generation** - Transforms user messages into a structured research brief
3. **Supervisor** - A lead researcher that analyzes the brief and delegates research tasks
4. **Parallel Researchers** - Multiple sub-agents that conduct focused research simultaneously
5. **Research Compression** - Each researcher synthesizes their findings
6. **Final Report** - All findings are combined into a comprehensive report

![Architecture Diagram](https://i.imgur.com/Q8HEZn0.png)

This differs from a section-based approach by allowing dynamic task decomposition based on the research question, rather than predefined sections.

---

# 🤝 Breakout Room #1
## Deep Research Foundations

In this breakout room, we'll understand the architecture and components of the Open Deep Research system.

## Task 1: Dependencies

You'll need API keys for Anthropic (for the LLM) and Tavily (for web search). We'll configure the system to use Anthropic's Claude Sonnet 4 exclusively.

In [2]:
import os
import getpass

os.environ["ANTHROPIC_API_KEY"] = getpass.getpass("Enter your Anthropic API key: ")
os.environ["TAVILY_API_KEY"] = getpass.getpass("Enter your Tavily API key: ")

## Task 2: State Definitions

The state structure is hierarchical with three levels:

### Agent State (Top Level)
Contains the overall conversation messages, research brief, accumulated notes, and final report.

### Supervisor State (Middle Level)
Manages the research supervisor's messages, research iterations, and coordinating parallel researchers.

### Researcher State (Bottom Level)
Each individual researcher has their own message history, tool call iterations, and research findings.

We also have structured outputs for tool calling:
- **ConductResearch** - Tool for supervisor to delegate research to a sub-agent
- **ResearchComplete** - Tool to signal research phase is done
- **ClarifyWithUser** - Structured output for asking clarifying questions
- **ResearchQuestion** - Structured output for the research brief

Let's import these from our library: [`open_deep_library/state.py`](open_deep_library/state.py)

In [1]:
# Import state definitions from the library
from open_deep_library.state import (
    # Main workflow states
    AgentState,           # Lines 65-72: Top-level agent state with messages, research_brief, notes, final_report
    AgentInputState,      # Lines 62-63: Input state is just messages
    
    # Supervisor states
    SupervisorState,      # Lines 74-81: Supervisor manages research delegation and iterations
    
    # Researcher states
    ResearcherState,      # Lines 83-90: Individual researcher with messages and tool iterations
    ResearcherOutputState, # Lines 92-96: Output from researcher (compressed research + raw notes)
    
    # Structured outputs for tool calling
    ConductResearch,      # Lines 15-19: Tool for delegating research to sub-agents
    ResearchComplete,     # Lines 21-22: Tool to signal research completion
    ClarifyWithUser,      # Lines 30-41: Structured output for user clarification
    ResearchQuestion,     # Lines 43-48: Structured output for research brief
)

## Task 3: Utility Functions and Tools

The system uses several key utilities:

### Search Tools
- **tavily_search** - Async web search with automatic summarization to stay within token limits
- Supports Anthropic native web search and Tavily API

### Reflection Tools
- **think_tool** - Allows researchers to reflect on their progress and plan next steps (ReAct pattern)

### Helper Utilities
- **get_all_tools** - Assembles the complete toolkit (search + MCP + reflection)
- **get_today_str** - Provides current date context for research
- Token limit handling utilities for graceful degradation

These are defined in [`open_deep_library/utils.py`](open_deep_library/utils.py)

In [3]:
# Import utility functions and tools from the library
from open_deep_library.utils import (
    # Search tool - Lines 43-136: Tavily search with automatic summarization
    tavily_search,
    
    # Reflection tool - Lines 219-244: Strategic thinking tool for ReAct pattern
    think_tool,
    
    # Tool assembly - Lines 569-597: Get all configured tools
    get_all_tools,
    
    # Date utility - Lines 872-879: Get formatted current date
    get_today_str,
    
    # Supporting utilities for error handling
    get_api_key_for_model,          # Lines 892-914: Get API keys from config or env
    is_token_limit_exceeded,         # Lines 665-701: Detect token limit errors
    get_model_token_limit,           # Lines 831-846: Look up model's token limit
    remove_up_to_last_ai_message,    # Lines 848-866: Truncate messages for retry
    anthropic_websearch_called,      # Lines 607-637: Detect Anthropic native search usage
    openai_websearch_called,         # Lines 639-658: Detect OpenAI native search usage
    get_notes_from_tool_calls,       # Lines 599-601: Extract notes from tool messages
)

## Task 4: Configuration System

The configuration system controls:

### Research Behavior
- **allow_clarification** - Whether to ask clarifying questions before research
- **max_concurrent_research_units** - How many parallel researchers can run (default: 5)
- **max_researcher_iterations** - How many times supervisor can delegate research (default: 6)
- **max_react_tool_calls** - Tool call limit per researcher (default: 10)

### Model Configuration
- **research_model** - Model for research and supervision (we'll use Anthropic)
- **compression_model** - Model for synthesizing findings
- **final_report_model** - Model for writing the final report
- **summarization_model** - Model for summarizing web search results

### Search Configuration
- **search_api** - Which search API to use (ANTHROPIC, TAVILY, or NONE)
- **max_content_length** - Character limit before summarization

Defined in [`open_deep_library/configuration.py`](open_deep_library/configuration.py)

In [4]:
# Import configuration from the library
from open_deep_library.configuration import (
    Configuration,    # Lines 38-247: Main configuration class with all settings
    SearchAPI,        # Lines 11-17: Enum for search API options (ANTHROPIC, TAVILY, NONE)
)

## Task 5: Prompt Templates

The system uses carefully engineered prompts for each phase:

### Phase 1: Clarification
**clarify_with_user_instructions** - Analyzes if the research scope is clear or needs clarification

### Phase 2: Research Brief
**transform_messages_into_research_topic_prompt** - Converts user messages into a detailed research brief

### Phase 3: Supervisor
**lead_researcher_prompt** - System prompt for the supervisor that manages delegation strategy

### Phase 4: Researcher
**research_system_prompt** - System prompt for individual researchers conducting focused research

### Phase 5: Compression
**compress_research_system_prompt** - Prompt for synthesizing research findings without losing information

### Phase 6: Final Report
**final_report_generation_prompt** - Comprehensive prompt for writing the final report

All prompts are defined in [`open_deep_library/prompts.py`](open_deep_library/prompts.py)

In [5]:
# Import prompt templates from the library
from open_deep_library.prompts import (
    clarify_with_user_instructions,                    # Lines 3-41: Ask clarifying questions
    transform_messages_into_research_topic_prompt,     # Lines 44-77: Generate research brief
    lead_researcher_prompt,                            # Lines 79-136: Supervisor system prompt
    research_system_prompt,                            # Lines 138-183: Researcher system prompt
    compress_research_system_prompt,                   # Lines 186-222: Research compression prompt
    final_report_generation_prompt,                    # Lines 228-308: Final report generation
)

## ❓ Question #1:

Explain the interrelationships between the three states (Agent, Supervisor, Researcher). Why don't we just make a single huge state?

##### Answer:
The Agent State has the original query aka goal, and the research brief in its state. It also holds the final report. The Agent state holds the overall workflow.

The Supervisor delegates tasks to different research agents, and hence its state is limited to subagent spawning - how many research subagents; what does each research on and the output from each of those subagents.
The Supervisor reads the plan or research brief from Agent state and writes to it the merged output from different Researchers.  

The Researcher state is only limited to its own specific task, and output messages. It gets the research topic from Supervisor, and has its own message list.


This separation allows for proper context management, avoid context bloat where each is limited to what it needs to perform its tasks/responsibilities. Also, this allows for parallelization - as each Researcher instance can act independently with clean state.
Having a single state would make it confusing for different subgraphs and also hard to debug.

## ❓ Question #2:

What are the advantages and disadvantages of importing these components instead of including them in the notebook?

##### Answer:
The key advantage of importing these components is - it makes it easier to understand the high level architecture and follow the flow. As the system has many components, having all the code within the notebook makes it really lengthy and harder to navigate.

One disadvantage I can think of is that - it's not immediately apparent what each of these components actually do (though descriptions are helpful, they are not enough to understand the technical details). So, have to keep switching between the notebook and the code files. :)

## 🏗️ Activity #1: Explore the Prompts

Open `open_deep_library/prompts.py` and examine one of the prompt templates in detail.

**Requirements:**
1. Choose one prompt template (clarify, brief, supervisor, researcher, compression, or final report)
2. Explain what the prompt is designed to accomplish
3. Identify 2-3 key techniques used in the prompt (e.g., structured output, role definition, examples)
4. Suggest one improvement you might make to the prompt

**YOUR CODE HERE** - Write your analysis in a markdown cell below

#### Chosen prompt: final_report_generation_prompt

**What the prompt is designed to accomplish**

The final report generation prompt is designed to generate the final research report to answer the user query by leveraging the following:
- The research brief
- Messages from Supervisor subgraph (which is the merged output from one or more Researcher subagents)

**Key techniques used in the prompt**

The prompt includes instructions and guidelines for formatting the report (sections, headings, paragraphs, lists etc.) appropriately based on the research question; Gives some examples (e.g. compare A vs B, list, overview), but also gives flexibility in not limiting the format to those examples.

The prompt also defines rules for the report language - that it should match the language of the user query, even if the research and internal messages are in Englih.

The prompt also includes Citations rules, and includes other restrictions such as "no self-referencing".

**Suggested improvements**

I can add a rule to limit the number of words or pages in the final report, so that it's not too short or lengthy.
Similarly, I can add a formatting instruction to include tables of comparison when the research output includes different aspects to compare (tools, approaches etc.).
Also, another improvement is to prompt it to add a section at the end to highlight the limitations and gaps in this research. 


---

# 🤝 Breakout Room #2
## Building & Running the Researcher

In this breakout room, we'll explore the node functions, build the graph, and run wellness research.

## Task 6: Node Functions - The Building Blocks

Now let's look at the node functions that make up our graph. We'll import them from the library and understand what each does.

### The Complete Research Workflow

The workflow consists of 8 key nodes organized into 3 subgraphs:

1. **Main Graph Nodes:**
   - `clarify_with_user` - Entry point that checks if clarification is needed
   - `write_research_brief` - Transforms user input into structured research brief
   - `final_report_generation` - Synthesizes all research into final report

2. **Supervisor Subgraph Nodes:**
   - `supervisor` - Lead researcher that plans and delegates
   - `supervisor_tools` - Executes supervisor's tool calls (delegation, reflection)

3. **Researcher Subgraph Nodes:**
   - `researcher` - Individual researcher conducting focused research
   - `researcher_tools` - Executes researcher's tool calls (search, reflection)
   - `compress_research` - Synthesizes researcher's findings

All nodes are defined in [`open_deep_library/deep_researcher.py`](open_deep_library/deep_researcher.py)

### Node 1: clarify_with_user

**Purpose:** Analyzes user messages and asks clarifying questions if the research scope is unclear.

**Key Steps:**
1. Check if clarification is enabled in configuration
2. Use structured output to analyze if clarification is needed
3. If needed, end with a clarifying question for the user
4. If not needed, proceed to research brief with verification message

**Implementation:** [`open_deep_library/deep_researcher.py` lines 60-115](open_deep_library/deep_researcher.py#L60-L115)

In [6]:
# Import the clarify_with_user node
from open_deep_library.deep_researcher import clarify_with_user

### Node 2: write_research_brief

**Purpose:** Transforms user messages into a structured research brief for the supervisor.

**Key Steps:**
1. Use structured output to generate detailed research brief from messages
2. Initialize supervisor with system prompt and research brief
3. Set up supervisor messages with proper context

**Why this matters:** A well-structured research brief helps the supervisor make better delegation decisions.

**Implementation:** [`open_deep_library/deep_researcher.py` lines 118-175](open_deep_library/deep_researcher.py#L118-L175)

In [7]:
# Import the write_research_brief node
from open_deep_library.deep_researcher import write_research_brief

### Node 3: supervisor

**Purpose:** Lead research supervisor that plans research strategy and delegates to sub-researchers.

**Key Steps:**
1. Configure model with three tools:
   - `ConductResearch` - Delegate research to a sub-agent
   - `ResearchComplete` - Signal that research is done
   - `think_tool` - Strategic reflection before decisions
2. Generate response based on current context
3. Increment research iteration count
4. Proceed to tool execution

**Decision Making:** The supervisor uses `think_tool` to reflect before delegating research, ensuring thoughtful decomposition of the research question.

**Implementation:** [`open_deep_library/deep_researcher.py` lines 178-223](open_deep_library/deep_researcher.py#L178-L223)

In [8]:
# Import the supervisor node (from supervisor subgraph)
from open_deep_library.deep_researcher import supervisor

### Node 4: supervisor_tools

**Purpose:** Executes the supervisor's tool calls, including strategic thinking and research delegation.

**Key Steps:**
1. Check exit conditions:
   - Exceeded maximum iterations
   - No tool calls made
   - `ResearchComplete` called
2. Process `think_tool` calls for strategic reflection
3. Execute `ConductResearch` calls in parallel:
   - Spawn researcher subgraphs for each delegation
   - Limit to `max_concurrent_research_units` (default: 5)
   - Gather all results asynchronously
4. Aggregate findings and return to supervisor

**Parallel Execution:** This is where the magic happens - multiple researchers work simultaneously on different aspects of the research question.

**Implementation:** [`open_deep_library/deep_researcher.py` lines 225-349](open_deep_library/deep_researcher.py#L225-L349)

In [9]:
# Import the supervisor_tools node
from open_deep_library.deep_researcher import supervisor_tools

### Node 5: researcher

**Purpose:** Individual researcher that conducts focused research on a specific topic.

**Key Steps:**
1. Load all available tools (search, MCP, reflection)
2. Configure model with tools and researcher system prompt
3. Generate response with tool calls
4. Increment tool call iteration count

**ReAct Pattern:** Researchers use `think_tool` to reflect after each search, deciding whether to continue or provide their answer.

**Available Tools:**
- Search tools (Tavily or Anthropic native search)
- `think_tool` for strategic reflection
- `ResearchComplete` to signal completion
- MCP tools (if configured)

**Implementation:** [`open_deep_library/deep_researcher.py` lines 365-424](open_deep_library/deep_researcher.py#L365-L424)

In [10]:
# Import the researcher node (from researcher subgraph)
from open_deep_library.deep_researcher import researcher

### Node 6: researcher_tools

**Purpose:** Executes the researcher's tool calls, including searches and strategic reflection.

**Key Steps:**
1. Check early exit conditions (no tool calls, native search used)
2. Execute all tool calls in parallel:
   - Search tools fetch and summarize web content
   - `think_tool` records strategic reflections
   - MCP tools execute external integrations
3. Check late exit conditions:
   - Exceeded `max_react_tool_calls` (default: 10)
   - `ResearchComplete` called
4. Continue research loop or proceed to compression

**Error Handling:** Safely handles tool execution errors and continues with available results.

**Implementation:** [`open_deep_library/deep_researcher.py` lines 435-509](open_deep_library/deep_researcher.py#L435-L509)

In [11]:
# Import the researcher_tools node
from open_deep_library.deep_researcher import researcher_tools

### Node 7: compress_research

**Purpose:** Compresses and synthesizes research findings into a concise, structured summary.

**Key Steps:**
1. Configure compression model
2. Add compression instruction to messages
3. Attempt compression with retry logic:
   - If token limit exceeded, remove older messages
   - Retry up to 3 times
4. Extract raw notes from tool and AI messages
5. Return compressed research and raw notes

**Why Compression?** Researchers may accumulate lots of tool outputs and reflections. Compression ensures:
- All important information is preserved
- Redundant information is deduplicated
- Content stays within token limits for the final report

**Token Limit Handling:** Gracefully handles token limit errors by progressively truncating messages.

**Implementation:** [`open_deep_library/deep_researcher.py` lines 511-585](open_deep_library/deep_researcher.py#L511-L585)

In [12]:
# Import the compress_research node
from open_deep_library.deep_researcher import compress_research

### Node 8: final_report_generation

**Purpose:** Generates the final comprehensive research report from all collected findings.

**Key Steps:**
1. Extract all notes from completed research
2. Configure final report model
3. Attempt report generation with retry logic:
   - If token limit exceeded, truncate findings by 10%
   - Retry up to 3 times
4. Return final report or error message

**Token Limit Strategy:**
- First retry: Use model's token limit × 4 as character limit
- Subsequent retries: Reduce by 10% each time
- Graceful degradation with helpful error messages

**Report Quality:** The prompt guides the model to create well-structured reports with:
- Proper headings and sections
- Inline citations
- Comprehensive coverage of all findings
- Sources section at the end

**Implementation:** [`open_deep_library/deep_researcher.py` lines 607-697](open_deep_library/deep_researcher.py#L607-L697)

In [13]:
# Import the final_report_generation node
from open_deep_library.deep_researcher import final_report_generation

## Task 7: Graph Construction - Putting It All Together

The system is organized into three interconnected graphs:

### 1. Researcher Subgraph (Bottom Level)
Handles individual focused research on a specific topic:
```
START → researcher → researcher_tools → compress_research → END
               ↑            ↓
               └────────────┘ (loops until max iterations or ResearchComplete)
```

### 2. Supervisor Subgraph (Middle Level)
Manages research delegation and coordination:
```
START → supervisor → supervisor_tools → END
            ↑              ↓
            └──────────────┘ (loops until max iterations or ResearchComplete)
            
supervisor_tools spawns multiple researcher_subgraphs in parallel
```

### 3. Main Deep Researcher Graph (Top Level)
Orchestrates the complete research workflow:
```
START → clarify_with_user → write_research_brief → research_supervisor → final_report_generation → END
                 ↓                                       (supervisor_subgraph)
               (may end early if clarification needed)
```

Let's import the compiled graphs from the library.

In [14]:
# Import the pre-compiled graphs from the library
from open_deep_library.deep_researcher import (
    # Bottom level: Individual researcher workflow
    researcher_subgraph,    # Lines 588-605: researcher → researcher_tools → compress_research
    
    # Middle level: Supervisor coordination
    supervisor_subgraph,    # Lines 351-363: supervisor → supervisor_tools (spawns researchers)
    
    # Top level: Complete research workflow
    deep_researcher,        # Lines 699-719: Main graph with all phases
)

## Why This Architecture?

### Advantages of Supervisor-Researcher Delegation

1. **Dynamic Task Decomposition**
   - Unlike section-based approaches with predefined structure, the supervisor can break down research based on the actual question
   - Adapts to different types of research (comparisons, lists, deep dives, etc.)

2. **Parallel Execution**
   - Multiple researchers work simultaneously on different aspects
   - Much faster than sequential section processing
   - Configurable parallelism (1-20 concurrent researchers)

3. **ReAct Pattern for Quality**
   - Researchers use `think_tool` to reflect after each search
   - Prevents excessive searching and improves search quality
   - Natural stopping conditions based on information sufficiency

4. **Flexible Tool Integration**
   - Easy to add MCP tools for specialized research
   - Supports multiple search APIs (Anthropic, Tavily)
   - Each researcher can use different tool combinations

5. **Graceful Token Limit Handling**
   - Compression prevents token overflow
   - Progressive truncation in final report generation
   - Research can scale to arbitrary depths

### Trade-offs

- **Complexity:** More moving parts than section-based approach
- **Cost:** Parallel researchers use more tokens (but faster)
- **Unpredictability:** Research structure emerges dynamically

## Task 8: Running the Deep Researcher

Now let's see the system in action! We'll use it to research wellness strategies for improving sleep quality.

### Setup

We need to:
1. Set up the wellness research request
2. Configure the execution with Anthropic settings
3. Run the research workflow

In [15]:
# Set up the graph with Anthropic configuration
from IPython.display import Markdown, display
import uuid

# Note: deep_researcher is already compiled from the library
# For this demo, we'll use it directly without additional checkpointing
graph = deep_researcher

print("✓ Graph ready for execution")
print("  (Note: The graph is pre-compiled from the library)")

✓ Graph ready for execution
  (Note: The graph is pre-compiled from the library)


### Configuration for Anthropic

We'll configure the system to use:
- **Claude Sonnet 4** for all research, supervision, and report generation
- **Tavily** for web search (you can also use Anthropic's native search)
- **Moderate parallelism** (1 concurrent researcher for cost control)
- **Clarification enabled** (will ask if research scope is unclear)

In [18]:
# Configure for Anthropic with moderate settings
config = {
    "configurable": {
        # Model configuration - using Claude Sonnet 4 for everything
        "research_model": "anthropic:claude-sonnet-4-20250514",
        "research_model_max_tokens": 10000,
        
        "compression_model": "anthropic:claude-sonnet-4-20250514",
        "compression_model_max_tokens": 8192,
        
        "final_report_model": "anthropic:claude-sonnet-4-20250514",
        "final_report_model_max_tokens": 10000,
        
        "summarization_model": "anthropic:claude-sonnet-4-20250514",
        "summarization_model_max_tokens": 8192,
        
        # Research behavior
        "allow_clarification": True,
        "max_concurrent_research_units": 1,  # 1 parallel researcher
        "max_researcher_iterations": 2,      # Supervisor can delegate up to 2 times
        "max_react_tool_calls": 3,           # Each researcher can make up to 3 tool calls
        
        # Search configuration
        "search_api": "tavily",  # Using Tavily for web search
        "max_content_length": 30000,
        
        # Thread ID for this conversation
        "thread_id": str(uuid.uuid4())
    }
}

print("✓ Configuration ready")
print(f"  - Research Model: Claude Sonnet 4")
print(f"  - Max Concurrent Researchers: 1")
print(f"  - Max Iterations: 2")
print(f"  - Search API: Tavily")

✓ Configuration ready
  - Research Model: Claude Sonnet 4
  - Max Concurrent Researchers: 1
  - Max Iterations: 2
  - Search API: Tavily


### Execute the Wellness Research

Now let's run the research! We'll ask the system to research evidence-based strategies for improving sleep quality.

The workflow will:
1. **Clarify** - Check if the request is clear (may skip if obvious)
2. **Research Brief** - Transform our request into a structured brief
3. **Supervisor** - Plan research strategy and delegate to researchers
4. **Parallel Research** - Researchers gather information simultaneously
5. **Compression** - Each researcher synthesizes their findings
6. **Final Report** - All findings combined into comprehensive report

In [19]:
# Create our wellness research request
research_request = """
I want to improve my sleep quality. I currently:
- Go to bed at inconsistent times (10pm-1am)
- Use my phone in bed
- Often feel tired in the morning

Please research the best evidence-based strategies for improving sleep quality and create a comprehensive sleep improvement plan for me.
"""

# Execute the graph
async def run_research():
    """Run the research workflow and display results."""
    print("Starting research workflow...\n")
    
    async for event in graph.astream(
        {"messages": [{"role": "user", "content": research_request}]},
        config,
        stream_mode="updates"
    ):
        # Display each step
        for node_name, node_output in event.items():
            print(f"\n{'='*60}")
            print(f"Node: {node_name}")
            print(f"{'='*60}")
            
            if node_name == "clarify_with_user":
                if "messages" in node_output:
                    last_msg = node_output["messages"][-1]
                    print(f"\n{last_msg.content}")
            
            elif node_name == "write_research_brief":
                if "research_brief" in node_output:
                    print(f"\nResearch Brief Generated:")
                    print(f"{node_output['research_brief'][:500]}...")
            
            elif node_name == "supervisor":
                print(f"\nSupervisor planning research strategy...")
                if "supervisor_messages" in node_output:
                    last_msg = node_output["supervisor_messages"][-1]
                    if hasattr(last_msg, 'tool_calls') and last_msg.tool_calls:
                        print(f"Tool calls: {len(last_msg.tool_calls)}")
                        for tc in last_msg.tool_calls:
                            print(f"  - {tc['name']}")
            
            elif node_name == "supervisor_tools":
                print(f"\nExecuting supervisor's tool calls...")
                if "notes" in node_output:
                    print(f"Research notes collected: {len(node_output['notes'])}")
            
            elif node_name == "final_report_generation":
                if "final_report" in node_output:
                    print(f"\n" + "="*60)
                    print("FINAL REPORT GENERATED")
                    print("="*60 + "\n")
                    display(Markdown(node_output["final_report"]))
    
    print("\n" + "="*60)
    print("Research workflow completed!")
    print("="*60)

# Run the research
await run_research()

Starting research workflow...


Node: clarify_with_user

I have sufficient information to proceed with your sleep improvement research. I understand you're looking for evidence-based strategies to address your current sleep challenges: inconsistent bedtimes (10pm-1am), phone use in bed, and morning fatigue. I will now research the best scientific approaches to improve sleep quality and create a comprehensive, personalized sleep improvement plan for you.

Node: write_research_brief

Research Brief Generated:
I want to improve my sleep quality based on evidence-based strategies. My current sleep challenges include: going to bed at inconsistent times (ranging from 10pm to 1am), using my phone in bed, and often feeling tired in the morning despite getting sleep. Please research the most effective, scientifically-backed methods for improving sleep quality that specifically address inconsistent sleep schedules, electronic device use before bed, and morning fatigue. I need a comprehensive sle


Node: research_supervisor

Node: final_report_generation

FINAL REPORT GENERATED



Error generating final report: Error code: 429 - {'type': 'error', 'error': {'type': 'rate_limit_error', 'message': "This request would exceed your organization's rate limit of 30,000 input tokens per minute (org: 82c83cc5-0db7-4b69-982f-84bef6590c6d, model: claude-sonnet-4-20250514). For details, refer to: https://docs.claude.com/en/api/rate-limits. You can see the response headers for current usage. Please reduce the prompt length or the maximum tokens requested, or try again later. You may also contact sales at https://www.anthropic.com/contact-sales to discuss your options for a rate limit increase."}, 'request_id': 'req_011CY1fXeJvKYhHv8RvQwtXe'}


Research workflow completed!


## Task 9: Understanding the Output

Let's break down what happened:

### Phase 1: Clarification
The system checked if your request was clear. Since you provided specific details about your sleep issues, it likely proceeded without asking clarifying questions.

### Phase 2: Research Brief
Your request was transformed into a detailed research brief that guides the supervisor's delegation strategy.

### Phase 3: Supervisor Delegation
The supervisor analyzed the brief and decided how to break down the research:
- Used `think_tool` to plan strategy
- Called `ConductResearch` to delegate to researchers
- Each delegation specified a focused research topic (e.g., sleep hygiene, circadian rhythm, blue light effects)

### Phase 4: Parallel Research
Researchers worked on their assigned topics:
- Each researcher used web search tools to gather information
- Used `think_tool` to reflect after each search
- Decided when they had enough information
- Compressed their findings into clean summaries

### Phase 5: Final Report
All research findings were synthesized into a comprehensive sleep improvement plan with:
- Well-structured sections
- Evidence-based recommendations
- Practical action items
- Sources for further reading

## Task 10: Key Takeaways & Next Steps

### Architecture Benefits
1. **Dynamic Decomposition** - Research structure emerges from the question, not predefined
2. **Parallel Efficiency** - Multiple researchers work simultaneously
3. **ReAct Quality** - Strategic reflection improves search decisions
4. **Scalability** - Handles token limits gracefully through compression
5. **Flexibility** - Easy to add new tools and capabilities

### When to Use This Pattern
- **Complex research questions** that need multi-angle investigation
- **Comparison tasks** where parallel research on different topics is beneficial
- **Open-ended exploration** where structure should emerge dynamically
- **Time-sensitive research** where parallel execution speeds up results

### When to Use Section-Based Instead
- **Highly structured reports** with predefined format requirements
- **Template-based content** where sections are always the same
- **Sequential dependencies** where later sections depend on earlier ones
- **Budget constraints** where token efficiency is critical

### Extend the System
1. **Add MCP Tools** - Integrate specialized tools for your domain
2. **Custom Prompts** - Modify prompts for specific research types
3. **Different Models** - Try different Claude versions or mix models
4. **Persistence** - Use a real database for checkpointing instead of memory

### Learn More
- [LangGraph Documentation](https://langchain-ai.github.io/langgraph/)
- [Open Deep Research Repo](https://github.com/langchain-ai/open_deep_research)
- [Anthropic Claude Documentation](https://docs.anthropic.com/)
- [Tavily Search API](https://tavily.com/)

## ❓ Question #3:

What are the trade-offs of using parallel researchers vs. sequential research? When might you choose one approach over the other?

##### Answer:

Parallel researchers are helpful to complete the research is less time than sequential. Also they are applicable where the research topics are fairly independent and hence can be run in parallel. 

Sequential research is useful if the research subtopics have inherent dependencies and benefit from the output of the others.

Tradeoffs:
- While Parallel researchers allow for faster research completion, the architecture of the system is more complex than the sequential. Parallel researchers approach can result in unpredictable cost and needs to take care of possible rate limits 
- Sequential researcher design is simpler; but is specific to a particular sequence and not flexible for different kinds of research. Sequential researcher approach assumes an order, so can have a predictable token usage.

When to choose each:
- Choose parallel, for time-sentivite report generation tasks, and for flexible report types.
- Choose sequential, when the research task depends on a specific order, and for time-insensitive reports.




## ❓ Question #4:

How would you adapt this deep research architecture for a production wellness application? What additional components would you need?

##### Answer:
For a production wellness application, additional components that can added are:
- Long-term memory with user preferences, so that the research can be tailored to user's profile and preferences
- Persistence in a database
- Add monitoring and observability (using Langsmith)
- Limit the research sources to authoritative and credible sources only 
- Add guardrails to avoid producing harmful information.
- Add Filesystem tools for context management
- Implement async execution for the deep research agent

## 🏗️ Activity #2: Custom Wellness Research

Using what you've learned, run a custom wellness research task.

**Requirements:**
1. Create a wellness-related research question (exercise, nutrition, stress, etc.)
2. Modify the configuration for your use case
3. Run the research and analyze the output
4. Document what worked well and what could be improved

**Experiment ideas:**
- Research exercise routines for specific conditions (bad knee, lower back pain)
- Compare different stress management techniques
- Investigate nutrition strategies for specific goals
- Explore meditation and mindfulness research

**YOUR CODE HERE**

In [20]:
# YOUR CODE HERE
# Create your own wellness research request and run it

my_wellness_request = """
Does research support specific supplements (e.g. iron, antioxidants, or others) 
for performance or acclimatization at altitude?
"""

# Optionally modify the config
my_config = {
    "configurable": {
        "research_model": "anthropic:claude-sonnet-4-20250514",
        "research_model_max_tokens": 2048,
        "compression_model": "anthropic:claude-sonnet-4-20250514",
        "compression_model_max_tokens": 2048,
        "final_report_model": "anthropic:claude-sonnet-4-20250514",
        "final_report_model_max_tokens": 4096,
        "summarization_model": "anthropic:claude-sonnet-4-20250514",
        "summarization_model_max_tokens": 2048,
        "allow_clarification": True,
        "max_concurrent_research_units": 1,
        "max_researcher_iterations": 2,
        "max_react_tool_calls": 3,
        "search_api": "tavily",
        "max_content_length": 20000,
        "thread_id": str(uuid.uuid4())
    }
}

async def run_custom_research(request, config):
    """Run custom research with the given request and config."""
    # Create the graph
    graph = deep_researcher

    # Run the research
    async for event in graph.astream(
        {"messages": [{"role": "user", "content": request}]},
        config,
        stream_mode="updates"
    ):
        # Display each step
        for node_name, node_output in event.items():
            print(f"\n{'='*60}")
            print(f"Node: {node_name}")
            print(f"{'='*60}")
            
            if node_name == "clarify_with_user":
                if "messages" in node_output:
                    last_msg = node_output["messages"][-1]
                    print(f"\n{last_msg.content}")
            
            elif node_name == "write_research_brief":
                if "research_brief" in node_output:
                    print(f"\nResearch Brief Generated:")
                    print(f"{node_output['research_brief'][:500]}...")
            
            elif node_name == "supervisor":
                print(f"\nSupervisor planning research strategy...")
                if "supervisor_messages" in node_output:
                    last_msg = node_output["supervisor_messages"][-1]
                    if hasattr(last_msg, 'tool_calls') and last_msg.tool_calls:
                        print(f"Tool calls: {len(last_msg.tool_calls)}")
                        for tc in last_msg.tool_calls:
                            print(f"  - {tc['name']}")
            
            elif node_name == "supervisor_tools":
                print(f"\nExecuting supervisor's tool calls...")
                if "notes" in node_output:
                    print(f"Research notes collected: {len(node_output['notes'])}")
            
            elif node_name == "final_report_generation":
                if "final_report" in node_output:
                    print(f"\n" + "="*60)
                    print("FINAL REPORT GENERATED")
                    print("="*60 + "\n")
                    display(Markdown(node_output["final_report"]))
    print("\n" + "="*60)
    print("Research workflow completed!")
    print("="*60)


# Run your research
# await run_custom_research(my_wellness_request, my_config)

await run_custom_research(my_wellness_request, my_config)


Node: clarify_with_user

To provide you with the most relevant and comprehensive research on altitude supplements, I need to clarify a few key aspects of your request:

• **Altitude range**: Are you interested in supplements for specific altitude ranges (e.g., moderate altitude 2,500-8,000 ft, high altitude 8,000-14,000 ft, or extreme altitude >14,000 ft)?

• **Target population**: Are you looking for research on:
  - Athletes/sports performance at altitude?
  - General population traveling to altitude?
  - Military/occupational exposure?
  - All of the above?

• **Outcomes of interest**: Should I focus on:
  - Performance enhancement (physical/cognitive)?
  - Acclimatization speed/effectiveness?
  - Prevention of altitude sickness?
  - All altitude-related outcomes?

• **Research scope**: Would you like me to include:
  - Only randomized controlled trials?
  - All types of clinical studies?
  - Meta-analyses and systematic reviews?

This information will help me provide a more target

In [21]:
my_refinedwellness_request = """
Does research support specific supplements (e.g. iron, antioxidants, or others) 
for performance or acclimatization at altitude? 
Focus on high altitude trekking for general population and all altitude-related outcomes.
Inlude only randomised controlled trials.
"""

await run_custom_research(my_refinedwellness_request, my_config)


Node: clarify_with_user

I have sufficient information to proceed with your research request. I understand you want a systematic review of randomized controlled trials examining the effectiveness of specific supplements (iron, antioxidants, and others) for performance enhancement or acclimatization support during high altitude trekking in the general population. The focus will be on all altitude-related outcomes and will be limited to RCTs only. I will now begin researching the available evidence on supplement interventions for high altitude trekking performance and acclimatization.

Node: write_research_brief

Research Brief Generated:
I need a comprehensive systematic review examining the effectiveness of specific supplements for high altitude trekking performance and acclimatization in the general population, based exclusively on randomized controlled trials. Please investigate the following supplements and their evidence base: iron supplementation, antioxidants (including but not 


Node: research_supervisor

Node: final_report_generation

FINAL REPORT GENERATED



# Effectiveness of Supplements for High Altitude Trekking: A Systematic Review of Randomized Controlled Trials

## Overview

The evidence base for supplement effectiveness in high altitude trekking is limited but growing, with several randomized controlled trials examining various interventions for performance and acclimatization outcomes. A comprehensive network meta-analysis published in 2025 analyzed 20 randomized controlled trials involving 329 participants engaging in physical activities at altitudes above 1,500 meters, providing the most comprehensive evidence to date [1]. The analysis examined eight different dietary interventions including carbohydrate supplementation, iron, antioxidant-rich foods, nitrates, and herbal compounds, measuring outcomes such as maximal oxygen uptake (VO2max), heart rate, peripheral oxygen saturation (SpO2), and perceived exertion.

## Iron Supplementation

Iron supplementation has produced conflicting results in randomized controlled trials, with effectiveness appearing to vary by population and methodology. The biological rationale for iron supplementation at altitude centers on its role in hemoglobin production, erythropoiesis, and cellular oxygen sensing pathways, particularly the hypoxia-inducible factor (HIF) system.

### Positive Evidence from Peru

A randomized, double-blinded, placebo-controlled trial conducted in Peru demonstrated potential protective effects of intravenous iron supplementation against acute mountain sickness (AMS) [2]. Twenty-four healthy male volunteers from Lima received either intravenous iron(III)-hydroxide sucrose (200 mg) or saline placebo before rapidly ascending to 4,340 meters altitude at Cerro de Pasco.

The iron group experienced a 65% smaller increase in AMS symptoms compared to the placebo group (1.2 vs 3.4 points increase, p<0.05). Changes in ferritin levels correlated negatively with changes in AMS scores (R=-0.43, p<0.05), suggesting that higher iron stores were associated with less severe symptoms. The incidence of AMS was 42% in the iron group versus 67% in the placebo group, though this difference was not statistically significant due to the small sample size. Notably, no cases of severe AMS occurred in the iron group, compared to 17% in the placebo group.

Importantly, iron supplementation did not affect hematocrit or arterial oxygen saturation, suggesting the protective effect was independent of hemoglobin concentration and likely related to cellular oxygen sensing mechanisms rather than enhanced oxygen transport capacity.

### Negative Evidence from China

A separate randomized, double-blinded, placebo-controlled trial examining Chinese participants traveling from Beijing (~50m) to Lhasa (3,650m) found no significant protective effect of intravenous iron supplementation [3]. Forty-one participants received either intravenous iron sucrose (200 mg) or placebo before flying to altitude, with AMS assessed using the Lake Louise Scoring system for 5 days.

While the iron supplementation group showed a numerically lower AMS rate compared to controls (26.3% vs. 52.6% at 24 hours), this difference was not statistically significant (P=0.097). Throughout the 5-day study period, no significant differences in AMS rates were observed between groups, despite significantly higher serum iron concentrations and transferrin saturation rates in the iron group.

### Iron Supplementation Summary

The network meta-analysis ranked iron supplementation highest for improving heart rate (56.54%) and hematocrit (66.67%) responses at altitude [1]. The conflicting results between studies may reflect population differences, ascent profiles (gradual vs. rapid), or other methodological variations. Current evidence suggests iron supplementation may provide some benefit for altitude acclimatization, particularly in populations with baseline iron deficiency, but results are inconsistent across studies.

## Carbohydrate Supplementation

Carbohydrate supplementation demonstrated the strongest evidence for improving exercise performance at altitude in the network meta-analysis. Carbohydrate supplementation significantly improved VO2max compared to placebo (SMD = 1.13, 95% CrI: 0.25 to 2.05) and reduced rating of perceived exertion scores (MD = −0.77, 95% CrI: −1.83 to −0.09) with moderate-quality evidence [1].

The observed increase in VO2max exceeded the minimally clinically important difference of 1.0 ml/kg/min, suggesting clinical relevance for trekkers and mountaineers. The combination of carbohydrate and glutamine ranked highest for improving peripheral oxygen saturation (SpO2) with a SUCRA score of 84.54% and perceived exertion with a SUCRA score of 69.37%.

The International Olympic Committee (IOC) Nutrition Expert Group specifically recommends that athletes increase their intake of energy and carbohydrates during high-altitude training, supporting these findings from controlled trials.

## Nitrate Supplementation

Nitrate supplementation via beetroot juice showed promising results for maintaining exercise performance at altitude in a randomized controlled trial during a 20-day military trekking expedition [4]. Twenty-two healthy adults received either beetroot juice (~12.5 mmol nitrate per day) or control supplement starting 3 days before departure and continuing until reaching 4,800m altitude.

Beetroot juice significantly increased salivary nitrite levels (p=0.007), which serve as precursors to bioactive nitric oxide. Most importantly, while the control group experienced significant decline in fitness scores at 4,800m compared to baseline (p=0.003), the beetroot juice group showed no decline in fitness (p=0.26). Heart rate recovery following exercise was significantly prolonged in the control group at 4,800m (p<0.01) but remained unchanged in the beetroot juice group (p=0.61).

However, nitrate supplementation did not affect the occurrence of high altitude illness (p=1.00), suggesting its benefits are specific to exercise performance rather than altitude acclimatization or AMS prevention. The mechanism likely involves enhancement of the nitrate-nitrite-NO pathway, which regulates blood flow distribution, mitochondrial respiration, and muscle metabolism.

## Antioxidant Supplementation

The network meta-analysis identified antioxidant-rich foods as one of the eight dietary interventions studied in randomized controlled trials at altitude, though specific details on individual antioxidant compounds were limited in the available evidence [1]. The analysis did not provide separate effect sizes for specific antioxidant compounds such as vitamin C, vitamin E, or alpha-lipoic acid.

Current evidence suggests that while antioxidant supplementation is commonly used by altitude trekkers, the randomized controlled trial evidence base for specific antioxidant compounds remains limited. The theoretical rationale involves reducing oxidative stress associated with hypoxic exposure, but more targeted studies are needed to establish effectiveness for specific compounds and dosing protocols.

## Other Supplements with RCT Evidence

### Rhodiola and Cordyceps Combination

The network meta-analysis included rhodiola crenulata combined with cordyceps sinensis as one of the eight dietary interventions studied in randomized controlled trials, though specific effect sizes were not detailed for this combination [1]. These adaptogenic compounds have traditional use for altitude-related applications and represent an area of emerging research interest.

### High-Protein Diets and D-Aspartic Acid

High-protein dietary interventions and d-aspartic acid supplementation were also included in the network meta-analysis, representing additional approaches studied in randomized controlled trials for altitude applications [1]. However, these interventions ranked lower than carbohydrate supplementation and iron for most outcomes measured.

## Clinical Implications and Recommendations

Based on the available randomized controlled trial evidence, several supplements show promise for high altitude trekking applications:

**Strongest Evidence:**
- Carbohydrate supplementation for exercise performance (moderate-quality evidence)
- Carbohydrate plus glutamine combination for oxygen saturation and perceived exertion

**Moderate Evidence:**
- Iron supplementation for cardiovascular responses and hematocrit (conflicting results between populations)
- Nitrate supplementation for maintaining exercise performance (single well-designed study)

**Limited Evidence:**
- Antioxidant-rich foods (included in network analysis but limited specific data)
- Adaptogenic herbs like rhodiola and cordyceps combinations

## Study Limitations and Research Gaps

The current evidence base has several important limitations. Most studies involved relatively small sample sizes (ranging from 20-40 participants), which limits statistical power to detect clinically meaningful differences. The network meta-analysis included only 329 total participants across all 20 studies, highlighting the limited scope of research in this area [1].

Population characteristics varied significantly between studies, with the network analysis showing 80.17% male participants and significant geographic variation. The predominance of male subjects limits generalizability to female trekkers, who may have different physiological responses to altitude and supplement interventions.

Altitude exposure protocols varied considerably, from gradual ascent profiles to rapid transportation to altitude, making it difficult to compare results across studies. Study durations ranged from 10 days to 7 weeks, and 40.59% of participants exercised at altitudes above 3,500 meters, creating heterogeneity in exposure conditions.

Outcome measures were inconsistent across studies, with some focusing on acute mountain sickness prevention, others on exercise performance, and still others on physiological markers. This heterogeneity makes it challenging to draw definitive conclusions about supplement effectiveness for specific altitude-related outcomes.

## Future Research Directions

Several research priorities emerge from this systematic review. Large-scale, multi-center randomized controlled trials are needed to provide adequate statistical power for detecting clinically meaningful differences in altitude-related outcomes. These studies should include adequate representation of female participants and diverse populations.

Standardized outcome measures should be adopted across studies, including validated scales for acute mountain sickness, standardized exercise performance protocols, and consistent physiological monitoring. Long-term follow-up studies are needed to assess whether supplement benefits persist throughout extended altitude exposure.

Mechanistic studies combining supplement interventions with detailed physiological monitoring could help clarify the pathways through which effective supplements provide benefits. This could guide the development of more targeted interventions and optimal dosing protocols.

## Conclusion

The randomized controlled trial evidence for supplement effectiveness in high altitude trekking shows the strongest support for carbohydrate supplementation for exercise performance outcomes. Iron supplementation shows promise but with conflicting results between populations, while nitrate supplementation appears effective for maintaining exercise capacity but not for preventing altitude illness. The evidence base for specific antioxidant compounds remains limited despite their common use by trekkers.

Trekkers and mountaineers should prioritize evidence-based interventions, with carbohydrate supplementation having the strongest support for improving exercise performance at altitude. Iron supplementation may provide benefits, particularly for individuals with baseline deficiency, but results are inconsistent. Nitrate supplementation represents a promising intervention for maintaining exercise performance during altitude exposure.

Future research should focus on larger, more diverse study populations with standardized outcome measures to strengthen the evidence base for supplement recommendations in high altitude environments.

### Sources

[1] Frontiers in Nutrition Network Meta-Analysis: https://www.frontiersin.org/articles/10.3389/fnut.2025.1234567/full

[2] Iron Supplementation Peru Study: https://journals.lww.com/high-altitudemedicine/Abstract/2018/19030/Intravenous_Iron_Supplementation_Reduces.3.aspx

[3] Iron Supplementation China Study: https://www.ncbi.nlm.nih.gov/pmc/articles/PMC7456789/

[4] Nitrate Supplementation Military Study: https://www.sciencedirect.com/science/article/pii/S1089860320301234


Research workflow completed!
